# Проект

## Постановка задачи

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.
Нужно построить модель с предельно большим значением F1-меры. Это значение должно быть не меньше, чем 0.59. Помимо F1-меры необходимо посчитать значение метрики AUC-ROC.

## Изучение и подготовка данных

In [1]:
import pandas as pd
import os
import warnings
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, roc_auc_score, accuracy_score
from itertools import product, combinations
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier

pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)
is_need_learn = False

Откроем файл с данными и изучим его

In [2]:
def open_file(filename, indexcol=None):
    path_home = './datasets/' + filename
    path_server = '/datasets/' + filename
    path_full = filename
    if os.path.exists(path_home):
        return pd.read_csv(path_home, index_col=indexcol)
    elif os.path.exists(path_server):
        return pd.read_csv(path_server, index_col=indexcol)
    elif os.path.exists(path_full):
        return pd.read_csv(path_full, index_col=indexcol)
    else:
        raise FileNotFoundError("cannot find file " + filename)

In [3]:
data = open_file('/home/bloodless/study/projects/sprint-2-proj-3/datasets/Churn_Modelling.csv', 0)

In [4]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1.1+ MB


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


В таблице 10000 строк и 14 столбцов. Пропусков нет. Последний столбец (_Exitted_) будет использоваться как целевой признак, а все остальные, кроме столбцов _CustomerId_ и _Surname_ - как обычные признаки для обучения моделей.

In [5]:
print('Доля клиентов, перешедших на тариф ultra:', data[data['Exited'] == 1]['Exited'].count()/data['Exited'].count())
print('Доля клиентов, перешедших на тариф ultra:', data[data['Exited'] == 0]['Exited'].count()/data['Exited'].count())

Доля клиентов, перешедших на тариф ultra: 0.2037
Доля клиентов, перешедших на тариф ultra: 0.7963


Соотношение признаков примерно 1 к 4. Довольно ощутимая разница, которая может повлиять на обучение и результаты, если мы будем обучать модель ровно на наших данных без каких-либо изменений. Вероятно, нам придется отредактировать данные для более корректного обучения.

Преобразуем категориальные признаки в численные с помощью one-head encoding. Всего у нас 2 категориальных признака - столбец _Genre_ с двумя возможными значениями и столбец _Geography_ с тремя возможными значениями. Поэтому после преобразования мы удалим два старых столбца и сгенерируем три новых, что не приведет к раздуванию таблицы.

In [6]:
data_ohe = pd.get_dummies(data.loc[:, (data.columns != 'CustomerId') & (data.columns != 'Surname')], drop_first=True)
data_ohe.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
RowNumber,,,,,,,,,,,,
1,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
2,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
3,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
4,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
5,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


Определим обучающую, валидационную и тестовую выборки. Соотношение будет 3:1:1. Для каждой выборки сохраним соотношение признаков.

In [7]:
features_1 = data_ohe[data_ohe['Exited'] == 1].loc[:, data_ohe.columns != 'Exited']
target_1 = data_ohe[data_ohe['Exited'] == 1]['Exited']
features_0 = data_ohe[data_ohe['Exited'] == 0].loc[:, data_ohe.columns != 'Exited']
target_0 = data_ohe[data_ohe['Exited'] == 0]['Exited']

main_features_1, test_features_1, main_target_1, test_target_1 = train_test_split(features_1, target_1, test_size=0.2, random_state=12345)
main_features_0, test_features_0, main_target_0, test_target_0 = train_test_split(features_0, target_0, test_size=0.2, random_state=12345)

train_features_1, valid_features_1, train_target_1, valid_target_1 = train_test_split(main_features_1, main_target_1, test_size=0.25, random_state=12345)
train_features_0, valid_features_0, train_target_0, valid_target_0 = train_test_split(main_features_0, main_target_0, test_size=0.25, random_state=12345)

main_features = pd.concat([main_features_1, main_features_0])
train_features = pd.concat([train_features_1, train_features_0])
test_features = pd.concat([test_features_1, test_features_0])
valid_features = pd.concat([valid_features_1, valid_features_0])
main_target = pd.concat([main_target_1, main_target_0])
train_target = pd.concat([train_target_1, train_target_0])
test_target = pd.concat([test_target_1, test_target_0])
valid_target = pd.concat([valid_target_1, valid_target_0])

print('Train sample size', train_features.shape[0])
print('Test sample size', test_features.shape[0])
print('Test sample size', valid_features.shape[0])

Train sample size 5998
Test sample size 2001
Test sample size 2001


Определим массив числовых признаков и нормируем эти признаки. 

In [8]:
# столбцы 'HasCrCard' и 'IsActiveMember' численные, но имеют значения 0 и 1
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(train_features[numeric])

train_features[numeric] = scaler.transform(train_features[numeric])
valid_features[numeric] = scaler.transform(valid_features[numeric])

print(train_features.shape)
print(valid_features.shape)
display(train_features.head())
valid_features.head()

(5998, 11)
(2001, 11)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
RowNumber,,,,,,,,,,,
4730,0.345763,0.303605,-0.35107,1.143057,-0.914176,1,1,-0.967680,0,0,0
5389,-0.782942,1.456758,-1.38871,0.362816,-0.914176,0,0,-1.663127,1,0,1
1220,0.449314,0.495797,-0.00519,0.762143,0.805783,1,1,-0.927391,1,0,1
6233,0.780677,0.784085,-1.38871,0.445054,-0.914176,0,0,-0.234951,1,0,0
3338,-0.731166,1.648950,-0.35107,0.355841,-0.914176,0,0,-1.669634,0,1,1


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
RowNumber,,,,,,,,,,,
2696,-0.544775,0.015316,1.03245,1.372230,2.525741,1,0,-1.118685,1,0,0
5536,-0.016665,-0.657356,1.03245,1.285485,2.525741,1,0,1.588956,1,0,0
521,2.075063,-0.369068,-1.38871,2.164257,-0.914176,1,0,1.546660,0,0,0
9211,-2.771119,-0.272972,-1.73459,-1.241989,-0.914176,1,1,1.388801,0,1,1
7557,0.915293,0.976277,-0.69695,-1.241989,2.525741,1,1,-0.611645,0,1,1


Так как в выборке наблюдается значительный дизбаланс классов, создадим сбалансированные выборки - с помощью расширения и с помощью сужения исходной.

In [9]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [10]:
features_upsampled, target_upsampled = upsample(train_features, train_target, 4)
print('Upsampled train sample size', features_upsampled.shape[0])
print('Class 1', features_upsampled[target_upsampled == 1].shape[0])
print('Class 2', features_upsampled[target_upsampled == 0].shape[0])

Upsampled train sample size 9661
Class 1 4884
Class 2 4777


In [11]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [12]:
features_downsampled, target_downsampled = downsample(train_features, train_target, 0.26)
print('Downsampled train sample size', features_downsampled.shape[0])
print('Class 1', features_downsampled[target_downsampled == 1].shape[0])
print('Class 2', features_downsampled[target_downsampled == 0].shape[0])

Downsampled train sample size 2463
Class 1 1221
Class 2 1242


Теперь определим список всевозможных комбинаций признаков.

In [13]:
features_list = []

for i in range(11, 12):
    comb = list(combinations(train_features[2:12].columns, i))
    for a in comb:
        features_list.append(list(a))
    
len(features_list)

1

Мы подготовили данные для обучения моделей.